# Imports

In [1]:
import csv
import sklearn.linear_model
import sklearn.preprocessing
import sklearn.metrics
import sklearn.datasets
import numpy as np
import dataclasses
import matplotlib.pyplot as plt
from IPython.display import display
import ipywidgets
import typing
import abc
import math

In [2]:
def sigmoid(x: float):
    return 1 / (1 + math.exp(-x))

def rectified_linear(x: float):
    if x <= 0:
        return 0
    else:
        return x

# Data

In [3]:
def read_data_iris(path: str):
    with open(path) as f:
        reader = csv.reader(f)
        
        inp = []
        out = []
        for line in reader:
            if len(line) == 0:
                continue
            
            features_line = []
            inp.append(features_line)
            for i, value in enumerate(line):
                if i < 4:
                    value = float(value)
                    features_line.append(value)
                else:
                    out.append(value)
        
        inp = np.array(inp)
        out_name_to_number = {
            "Iris-setosa": 0,
            "Iris-versicolor": 1,
            "Iris-virginica": 2,
        }
        out = np.array([ out_name_to_number[name] for name in out ])
        
        return inp, out

data_inp_iris, data_out_iris = read_data_iris("data/iris.data")

In [4]:
def read_data_breast_cancer(path: str):
    with open(path) as f:
        reader = csv.reader(f)
        
        inp = []
        out = []
        for line in reader:
            if len(line) == 0:
                continue
            
            try:
                features_line = []
                for i, value in enumerate(line):
                    if i < 10:
                        value = float(value)
                        features_line.append(value)
                    else:
                        if value == "2":
                            out.append(0)
                        elif value == "4":
                            out.append(1)
                        else:
                            assert False
                inp.append(features_line)
            except ValueError:
                pass
        
        inp = np.array(inp)
        out = np.array(out)
        
        return inp, out

data_inp_breast_cancer, data_out_breast_cancer = read_data_breast_cancer("data/breast-cancer.data")

# Solution

In [19]:
class Regression(metaclass=abc.ABCMeta):
    @abc.abstractmethod
    def fit(self, inp: np.ndarray, out: np.ndarray):
        pass
    
    @abc.abstractmethod
    def predict(self, inp: np.ndarray) -> np.ndarray:
        pass

In [371]:
def cross_validate(
    inp: np.ndarray,
    out: np.ndarray,
    *,
    k: int,
    regressor: Regression,
):
    length = len(inp)

    # shuffle data
    indices = np.arange(length)
    np.random.shuffle(indices)

    inp = inp[indices]
    out = out[indices]
    
    accuracy_sum = 0
    
    negative_log_likelihood_loss = 0
    for i in range(k):
        negative_log_likelihood_loss_sum = 0
    
        # split into train and test
        test_start_index = int((i  )/k * length)
        test_stop_index =  int((i+1)/k * length)
        
        inp_test = inp[test_start_index:test_stop_index,:]
        out_test = out[test_start_index:test_stop_index]
        
        inp_train = np.delete(inp, np.s_[test_start_index:test_stop_index], 0)
        out_train = np.delete(out, np.s_[test_start_index:test_stop_index], 0)
        
        # normallize
        scaler = sklearn.preprocessing.StandardScaler()
        scaler.fit(inp_train)
        inp_train = scaler.transform(inp_train)
        inp_test = scaler.transform(inp_test)

        # train
        regressor.fit(inp_train, out_train)
        
        # test
        predicted_test, full_predictions = regressor.predict(inp_test)
        accuracy = sklearn.metrics.accuracy_score(out_test, predicted_test)
        accuracy_sum += accuracy
        
        # negative log likelihood loss
        for correct_class, prediction in zip(out_train, full_predictions):
            for prediction_value in prediction:
                if prediction_value[0] == correct_class:
                    correct_prediction = prediction_value
                    break
            negative_log_likelihood_loss_sum -= (
                math.log(correct_prediction[1])
            )
        negative_log_likelihood_loss += negative_log_likelihood_loss_sum
    
    accuracy = accuracy_sum / k
    negative_log_likelihood_loss = negative_log_likelihood_loss / k
    
    print(accuracy)
    print(negative_log_likelihood_loss)

In [21]:
def get_2d_shape(matrix: list[list[typing.Any]]) -> (int, int):
    width = None
    height = len(matrix)
    for row in matrix:
        if width is None:
            width = len(row)
        assert width == len(row)
    return (width, height)

In [22]:
class EpochKind(metaclass=abc.ABCMeta):
    pass

class EpochKindStochastic(EpochKind):
    pass

class EpochKindBatch(EpochKind):
    pass

In [225]:
class ActivationFunctionKind(metaclass=abc.ABCMeta):
    pass

class ActivationFunctionKindSoftmax(ActivationFunctionKind):
    pass

@dataclasses.dataclass(kw_only=True)
class ActivationFunctionKindSigmoid(ActivationFunctionKind):
    threshold: typing.Optional[float] = None

In [272]:
class MyLogisticRegression(Regression):
    def __init__(
        self,
        *,
        epoch_kind: EpochKind,
        activation_function_kind: ActivationFunctionKind,
        learning_rate: float,
        regularization_param: float,
        generations_count: int,
    ):
        self.__learning_rate = learning_rate
        self.__regularization_param = regularization_param
        self.__generations_count = generations_count
        self.__category_set = None
        self.__w = None
        self.__epoch_kind = epoch_kind
        self.__activation_function_kind = activation_function_kind
    
    def __category_gradient(
        self,
        inp_values,
        correct_category,
        category,
    ) -> float:
        if type(self.__activation_function_kind) is ActivationFunctionKindSoftmax:
            computed_out = (
                math.exp(
                    sum(
                        w_value * inp_value
                        for w_value, inp_value
                        in zip(self.__w[category], [1] + inp_values)
                    )
                )
                /
                sum(
                    math.exp(
                        sum(
                            w_value * inp_value
                            for w_value, inp_value
                            in zip(self.__w[category_2], [1] + inp_values)
                        )
                    )
                    for category_2 in self.__category_set
                )
            )
            
            err = (
                float(category == correct_category)
                -
                computed_out
            )
            
            return [
                -1 * err * inp_value
                +
                -1 * self.__regularization_param * 2 * w_value
                for inp_value, w_value in zip([1] + inp_values, self.__w[category])
            ]
        elif type(self.__activation_function_kind) is ActivationFunctionKindSigmoid:
            computed_out = sigmoid(
                sum(
                    w_value * inp_value
                    for w_value, inp_value
                    in zip(self.__w[category], [1] + inp_values)
                )
            )
            
            err = (
                float(category == correct_category)
                -
                computed_out
            )
            
            return [
                err * computed_out * (1 - computed_out) * inp_value
                for inp_value in [1] + inp_values
            ]
        else:
            assert False
    
    def __category_computed(
        self,
        inp_values,
        category,
    ) -> float:
        if type(self.__activation_function_kind) is ActivationFunctionKindSoftmax:
            denominator = (
                sum(
                    math.exp(
                        sum(
                            w_value * inp_value
                            for w_value, inp_value
                            in zip(self.__w[category_2], [1] + inp_values)
                        )
                    )
                    for category_2 in self.__category_set
                )
            )
            
            numerator = (
                math.exp(
                    sum(
                        w_value * inp_value
                        for w_value, inp_value
                        in zip(self.__w[category], [1] + inp_values)
                    )
                )
            )
            
            computed = (
                -math.log(numerator / denominator)
                -
                self.__regularization_param
                *
                sum(
                    w_value**2
                    for w_value in self.__w[category]
                )
            )
            
            return computed
        elif type(self.__activation_function_kind) is ActivationFunctionKindSigmoid:
            return sigmoid(
                sum(
                    w_value * inp_value
                    for w_value, inp_value
                    in zip(self.__w[category], [1] + inp_values)
                )
            )
        else:
            assert False
    
    def fit(self, inp: np.ndarray, out: np.ndarray):
        inp = inp.tolist()
        out = out.tolist()
        
        self.__category_set = set(out)
        
        inp_shape = get_2d_shape(inp)
        
        self.__w = {
            out_value: [
                0 for _ in range(1 + inp_shape[0])
            ]
            for out_value in self.__category_set
        }
        
        for generation_index in range(self.__generations_count):
            if type(self.__epoch_kind) is EpochKindBatch:
                batch_gradient_sum = {
                    out_value_2: [
                        0 for _ in range(1 + inp_shape[0])
                    ]
                    for out_value_2 in self.__category_set
                }
            
            for inp_values, correct_category in zip(inp, out):
                category_to_gradient = dict()
                for category in self.__category_set:
                    category_to_gradient[category] = self.__category_gradient(
                        inp_values,
                        correct_category,
                        category,
                    )
                
                if type(self.__epoch_kind) is EpochKindStochastic:
                    for out_value_2 in self.__category_set:                
                        for i in range(1 + inp_shape[0]):
                            self.__w[out_value_2][i] += category_to_gradient[out_value_2][i] * self.__learning_rate
                        
                if type(self.__epoch_kind) is EpochKindBatch:
                    for out_value_2 in self.__category_set:                
                        for i in range(1 + inp_shape[0]):
                            batch_gradient_sum[out_value_2][i] += category_to_gradient[out_value_2][i]
            
            if type(self.__epoch_kind) is EpochKindBatch:
                for out_value_2 in self.__category_set:
                    for i in range(1 + inp_shape[0]):
                        self.__w[out_value_2][i] += batch_gradient_sum[out_value_2][i] / inp_shape[1] * self.__learning_rate
    
    def predict(self, inp: np.ndarray) -> np.ndarray:
        inp = inp.tolist()
        
        full_predictions = []
        predictions = []
        for inp_values in inp:
            possibilities = []
            
            use_threshold = (
                type(self.__activation_function_kind) is ActivationFunctionKindSigmoid
                and
                self.__activation_function_kind.threshold is not None
            )
            
            negative_log_likelihood_loss = 0
            for category in self.__category_set:
                computed_out = self.__category_computed(
                    inp_values,
                    category,
                )
                
                if use_threshold:
                    computed_out = float(
                        computed_out > self.__activation_function_kind.threshold
                    )
                
                possibilities.append((category, computed_out))
            
            full_predictions.append(possibilities)
            
            if use_threshold:
                for possibility in possibilities:
                    if possibility[1] == 1:
                        best_possibility = possibility[0]
            else:
                best_possibility = max(possibilities, key=lambda a: a[1])[0]
            
            predictions.append(best_possibility)
        predictions = np.array(predictions)
        return predictions, full_predictions

In [222]:
class ToolLogisticRegression(Regression):
    def __init__(self):
        self.__regressor = sklearn.linear_model.LogisticRegression()
    
    def fit(self, inp: np.ndarray, out: np.ndarray):
        self.__regressor.fit(inp, out)
    
    def predict(self, inp: np.ndarray) -> np.ndarray:
        return self.__regressor.predict(inp)

## Breast cancer

In [223]:
cross_validate(
    data_inp_breast_cancer,
    data_out_breast_cancer,
    k=10,
    regressor=MyLogisticRegression(
        epoch_kind=EpochKindStochastic(),
        # epoch_kind=EpochKindBatch(),
        activation_function_kind=ActivationFunctionKindSoftmax(),
        learning_rate = 0.01,
        regularization_param = 0.007,
        generations_count = 1,
    ),
    # regressor=ToolLogisticRegression(),
)

0.9691602728047741


## Iris

In [384]:
cross_validate(
    data_inp_iris,
    data_out_iris,
    k=10,
    regressor=MyLogisticRegression(
        epoch_kind=EpochKindStochastic(),
        # epoch_kind=EpochKindBatch(),
        # activation_function_kind=ActivationFunctionKindSoftmax(),
        activation_function_kind=ActivationFunctionKindSigmoid(
            # threshold=0.01
        ),
        learning_rate = 0.05,
        regularization_param = 0.1,
        generations_count = 5,
    ),
    # regressor=ToolLogisticRegression(),
)

0.82
7.7564538843421875
